## Get Max Variables
This notebook takes raw daily data and output variables conditioned on maximum t2m across each year.

Have option to condition on daily mean or daily maximum `t2m` through the `t2m_av_use` variable. Zhang 2023 used daily maximum

In [1]:
import os
import xarray as xr
import numpy as np
from numba import jit
from tqdm.notebook import tqdm
from isca_tools.utils.moist_physics import sphum_from_dew
print(os.getcwd())

/home/users/jamd1/Isca


In [2]:
dir_base = '/gws/nopw/j04/global_ex/jamd1/era5/'
dir_t500 = os.path.join(dir_base, 'daily_mean', 't500')
dir_z500 = os.path.join(dir_base, 'daily_mean', 'z500')
dir_z500 = '/home/users/jamd1/Isca/jobs/era5/zhang_2023/raw/daily_mean/z500/'
dir_sp = os.path.join(dir_base, 'daily_mean', 'sp')
dir_d2m = os.path.join(dir_base, 'daily_mean', 'd2m')
dir_t2m = {key: os.path.join(dir_base, f'daily_{key}', 't2m') for key in ['mean', 'max']}

complevel = 4           # how much to compress by when saving

In [3]:
t2m_av_use = 'mean'     # get variables conditioned on annual maximum of daily max or mean temperature

In [4]:
@jit
def _at_t2m_max_ufunc(t2m,y):
    return y[np.argmax(t2m)]

def xr_at_t2m_max(t2m,y): # Find the level of a variable on the annual hottest day for each location
    return xr.apply_ufunc(_at_t2m_max_ufunc, t2m, y,
                          input_core_dims=[['time'],['time']], dask ='parallelized', vectorize=True, output_dtypes=[y.dtype])

In [5]:
def load_ds(path, rename_valid_time=True):
    # The t500 dataset has time as `valid_time` rather than `time`. Deal with this
    ds = xr.open_mfdataset(path)
    if rename_valid_time and 'valid_time' in ds.dims:
        ds = ds.rename({'valid_time': 'time'})
    return ds

In [6]:
# For each year, load in data, compute day with max t2m, and output t500 on this day as well as the index of this day
var_input_dir = {'t2m': dir_t2m[t2m_av_use], 't500': dir_t500, 'sp': dir_sp, 'd2m': dir_d2m, 'z500': dir_z500, 'day_ind': None}
var_id = {'t2m': 't2m', 't500': 't', 'sp': 'sp', 'd2m': 'd2m', 'z500': 'z'}    # name of var within input directory

var_out_file_name = {var: f"{var}_{'max' if var=='t2m' else 'of_t2m_max'}" for var in var_input_dir}
var_out_path = {key: f'/home/users/jamd1/Isca/jobs/era5/zhang_2023/processed/daily_{t2m_av_use}/{var_out_file_name[key]}.nc' for key in var_out_file_name}
var_out = {key: [] for key in var_out_path}
var_save = {var: True for var in var_out_path}
for var in var_save:
    if os.path.exists(var_out_path[var]):
        print(f'Data for {var} already exists, will skip')
        var_save[var] = False
    else:
        print(f"Data for {var} does not exist, will create it")
if all(not v for v in var_save.values()):
    raise ValueError('Data exists for all variables')

for year in tqdm(range(1979, 2022)):
    # Only load in t2m data, or data that needs to be saved, and that we have raw data for
    var_in_use = {key: load_ds(f"{var_input_dir[key]}/{year}.nc").chunk(dict(time=-1))[var_id[key]]
                  for key in var_input_dir if (key=='t2m' or var_save[key]) and (var_input_dir[key] is not None)}
    var_in_use['day_ind'] = np.arange(var_in_use['t2m'].time.size)   # varies as some years are leap years
    for key in var_out:
        if not var_save[key]:
            continue
        var_out_use = xr_at_t2m_max(var_in_use['t2m'], var_in_use[key]).compute()
        var_out_use = var_out_use.expand_dims(year=[year])
        var_out[key].append(var_out_use)

Data for t2m already exists, will skip
Data for t500 already exists, will skip
Data for sp already exists, will skip
Data for d2m already exists, will skip
Data for z500 does not exist, will create it
Data for day_ind already exists, will skip


  0%|          | 0/43 [00:00<?, ?it/s]

In [7]:
def set_attrs(var, long_name, units, description):
    # Function to set main attributes of given variable
    var.attrs['long_name'] = long_name
    var.attrs['units'] = units
    var.attrs['description'] = description
    return var

In [8]:
# Save data for all years in a single file for each variable
var_out_attrs = {'t2m': {'long_name': var_in_use['t2m'].long_name, 'units': var_in_use['t2m'].units,
                         'description': f'Max of daily {t2m_av_use} 2m temperature in a given year.'},
                 't500': {'long_name': '500hPa temperature', 'units': 'K',
                          'description': f'500 hPa temperature on day with maximum daily {t2m_av_use} 2m temperature in a given year.'},
                 'day_ind': {'long_name': 'Day index', 'units': 'Day',
                              'description': f'Day (0 is Jan 1st) with maximum daily {t2m_av_use} 2m temperature in a given year.'},
                 'sp': {'long_name': var_in_use['sp'].long_name, 'units': var_in_use['sp'].long_name,
                        'description': f'Surface pressure on day with maximum daily {t2m_av_use} 2m temperature in a given year.'},
                 'd2m': {'long_name': var_in_use['d2m'].long_name, 'units': var_in_use['d2m'].long_name,
                        'description': f'2m dew temperature on day with maximum daily {t2m_av_use} 2m temperature in a given year.'},
                 'z500': {'long_name': '500hPa geopotential', 'units': 'J/kg',
                          'description': f'500 hPa geopotential (gz) on day with maximum daily {t2m_av_use} 2m temperature in a given year.'}
                 }
for key in var_out_path:
    if not var_save[key]:
        continue
    if os.path.exists(var_out_path[key]):
        raise ValueError(f'Data exists for var={key} at\n{var_out_path[key]}')
    var_use = xr.concat(var_out[key], dim='year')
    if key == 'day_ind':
        # time ind has max value of 365 so can use int16
        var_use = var_use.astype('int16')
    var_use = set_attrs(var_use, var_out_attrs[key]['long_name'], var_out_attrs[key]['units'], var_out_attrs[key]['description'])
    var_use = xr.Dataset({key: var_use})    # make sure save as dataset rather than data array
    encoding = {var: {'zlib': True, 'complevel': complevel} for var in var_use.data_vars}
    var_use.to_netcdf(var_out_path[key], encoding=encoding)
    print(f'{key} data has been saved at:\n{var_out_path[key]}')

z500 data has been saved at:
/home/users/jamd1/Isca/jobs/era5/zhang_2023/processed/daily_mean/z500_of_t2m_max.nc


### Annual Mean 2m Specific Humidity
Below I compute the annual mean specific humidity at each grid point from the daily average values.

In [7]:
q2m_out = []
for year in tqdm(range(1979, 2022)):
    d2m_use = load_ds(f"{dir_d2m}/{year}.nc").d2m
    sp_use = load_ds(f"{dir_sp}/{year}.nc").sp
    q2m_use = sphum_from_dew(d2m_use, sp_use)
    q2m_use = q2m_use.mean(dim='time').compute()
    q2m_use = q2m_use.expand_dims(year=[year])
    q2m_out.append(q2m_use)

  0%|          | 0/43 [00:00<?, ?it/s]

In [10]:
var_use = xr.concat(q2m_out, dim='year')
var_use = set_attrs(var_use, '2m specific humidity', 'kg/kg', 'Annual mean 2m specific humidity')
var_use = xr.Dataset({'q2m': var_use})    # make sure save as dataset rather than data array
encoding = {var: {'zlib': True, 'complevel': complevel} for var in var_use.data_vars}
q2m_out_path = "/home/users/jamd1/Isca/jobs/era5/zhang_2023/processed/q2m_annual_mean.nc"
if os.path.exists(q2m_out_path):
    raise ValueError(f'Annual mean q2m data already exists at \n{q2m_out_path}')
else:
    var_use.to_netcdf(q2m_out_path, encoding=encoding)
    print(f'Annual mean q2m data has been saved at:\n{q2m_out_path}')

Annual mean q2m data has been saved at:
/home/users/jamd1/Isca/jobs/era5/zhang_2023/processed/q2m_annual_mean.nc
